In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_sp = pd.read_csv("../input/students-performance-in-exams/StudentsPerformance.csv")

In [ ]:
df_sp.head()

In [ ]:
def average_score(data):
    match_s = data[0]
    read_s = data[1]
    write_s = data[2]
    avg_score = (match_s + read_s + write_s)/3
    return avg_score

In [ ]:
# Adding a column to record the tweet_type based on replies, retweets and likes
df_sp['avg_score'] = pd.Series([0 for x in range(len(df_sp.index))])

In [ ]:
df_sp['avg_score'] = df_sp[['math score','reading score','writing score']].apply(average_score,axis=1)

In [ ]:
df_sp.head()

In [ ]:
plt.figure(figsize=[18,5])
plt.subplot(141)
sns.boxplot(y='math score', x = 'gender', data = df_sp)
plt.subplot(142)
sns.boxplot(y='reading score', x = 'gender',data = df_sp)
plt.subplot(143)
sns.boxplot(y='writing score', x = 'gender',data = df_sp)
plt.subplot(144)
sns.boxplot(y='avg_score', x = 'gender',data = df_sp)

In [ ]:
plt.figure(figsize=[18,5])
plt.subplot(141)
sns.distplot(df_sp['math score'])
plt.subplot(142)
sns.distplot(df_sp['reading score'])
plt.subplot(143)
sns.distplot(df_sp['writing score'])
plt.subplot(144)
sns.distplot(df_sp['avg_score'])

In [ ]:
plt.figure(figsize=[17,9])
plt.subplot(221)
sns.countplot(x='gender',data = df_sp)
plt.subplot(222)
sns.countplot(x='race/ethnicity',hue ='gender',data = df_sp)
plt.subplot(223)
sns.countplot(x='parental level of education',hue ='gender', data = df_sp)
plt.subplot(224)
sns.countplot(x='lunch',data = df_sp)

In [ ]:
df_sp.drop(['math score','reading score','writing score'],axis =1,inplace=True)

In [ ]:
df_sp.head()

In [ ]:
gender_var = pd.get_dummies(df_sp['gender'],drop_first=True)
df_sp.drop(['gender'], axis=1, inplace=True)
df_sp = pd.concat([df_sp, gender_var], axis=1)

In [ ]:
race_var = pd.get_dummies(df_sp['race/ethnicity'],drop_first=True)
df_sp.drop(['race/ethnicity'],axis=1,inplace=True)
df_sp = pd.concat([df_sp,race_var],axis=1)

In [ ]:
edu = pd.get_dummies(df_sp['parental level of education'],drop_first=True)
df_sp.drop(['parental level of education'],axis=1,inplace=True)
df_sp = pd.concat([df_sp,edu],axis = 1)

In [ ]:
lunch_var = pd.get_dummies(df_sp['lunch'],drop_first=True)
df_sp.drop(['lunch'],axis=1,inplace=True)
df_sp = pd.concat([df_sp,lunch_var],axis = 1)

In [ ]:
test_prep = pd.get_dummies(df_sp['test preparation course'],drop_first=True)
df_sp.drop(['test preparation course'],axis=1,inplace=True)
df_sp = pd.concat([df_sp,test_prep],axis = 1)

In [ ]:
df_sp.head()

In [ ]:
X = df_sp.drop('avg_score',axis=1).values
y = df_sp['avg_score'].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Lasso,Ridge,BayesianRidge,ElasticNet,HuberRegressor,LogisticRegression,SGDRegressor

In [ ]:
models = [ ['Linear Regression :', LinearRegression()],
           ['DecisionTree :',DecisionTreeRegressor()],
           ['RandomForest :',RandomForestRegressor(n_estimators = 300)],
           ['KNeighbours :', KNeighborsRegressor(n_neighbors = 5)],
           ['SVM :', SVR()],
           ['AdaBoostClassifier :', AdaBoostRegressor()],
           ['GradientBoostingClassifier: ', GradientBoostingRegressor()],
           ['Xgboost: ', XGBRegressor(max_depth = 6)],
           ['CatBoost: ', CatBoostRegressor(logging_level='Silent')],
           ['Lasso: ', Lasso()],
           ['Ridge: ', Ridge()],
           ['BayesianRidge: ', BayesianRidge()],
           ['ElasticNet: ', ElasticNet()]]

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.metrics import accuracy_score
MAE_score_model = {}
RMSE_score_model = {}
R2S_score_model = {}
ACC_score_model = {}
for name_of_model,model_parms in models:
    curr_model = model_parms
    curr_model.fit(X_train, y_train)
    curr_predict = curr_model.predict(X_test)
    MAE_score_model[name_of_model] = mean_absolute_error(y_test, curr_predict)
    RMSE_score_model[name_of_model] = np.sqrt(mean_squared_error(y_test, curr_predict))
    R2S_score_model[name_of_model] =r2_score(y_test, curr_predict)

In [ ]:
from collections import OrderedDict
MAE_sorted_scores_value = OrderedDict(sorted(MAE_score_model.items(), key=lambda x: x[1]))
RMSE_sorted_scores_value = OrderedDict(sorted(RMSE_score_model.items(), key=lambda x: x[1]))
R2S_sorted_scores_value = OrderedDict(sorted(R2S_score_model.items(), key=lambda x: x[1],reverse = True))

In [ ]:
print('MAE (Mean Absolute Error) Scores for the models are as follows :')
print('Tip: Ranges from 0 to infinity and lower value means better model')
MAE_sorted_scores_value

In [ ]:
print('RMSE (Root Mean Square Error) Scores for the models are as follows :')
print('Tip 1: It ranges from 0 to infinity and lower means better model and it is always greater in magnitude than MAE')
print('Tip 2: If diff between RMSE and MAE is more then it means there are more outliers in the data')
RMSE_sorted_scores_value

In [ ]:
print('R2S (R-Squared or Adj R-Squared Score) Scores for the models are as follows :')
print('Tip 1: It ranges from 0 to 1. Closer to 1 Better is the model')
R2S_sorted_scores_value

In [ ]:
# Lets plot one of the model and see the test and predicted data
regressor = LinearRegression()
regressor.fit(X_train,y_train)

In [ ]:
y_predict = regressor.predict(X_test)

In [ ]:
actual_vals_y_test = pd.DataFrame(data = y_test, columns = ['average_sal'])
predict_vals_y_pred = pd.DataFrame(data = y_predict, columns = ['average_sal'])

In [ ]:
plt.figure(figsize=[20,9])
sns.lineplot(x = actual_vals_y_test.index, y = actual_vals_y_test['average_sal'], data = actual_vals_y_test)
sns.lineplot(x = predict_vals_y_pred.index, y = predict_vals_y_pred['average_sal'], data = predict_vals_y_pred)